In [1]:
%load_ext autoreload
%autoreload 2

from superai.llm.agents import GeneralAgent
from superai.llm.data_types import Any
from superai.llm.memory import LocalMemory
from superai.llm.evaluate import Evaluator
from superai.llm.configuration import Configuration
from superai.llm.logger import logger
from superai.llm.dataset import Dataset
from superai.llm.foundation_models import ChatGPT
from superai.llm.tools import list_commands

config = Configuration()

/opt/homebrew/lib/python3.11/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


AttributeError: 'OpenAIEmbedding' object has no attribute '__fields_set__'

In [34]:
from colorama import Fore

DEBUG:  Warning: The file '/Users/bradcordova/Documents/GitHub/super_llm/db/superai.llm.json' already exists. Overwriting /Users/bradcordova/Documents/GitHub/super_llm/db/superai.llm.json


In [35]:
Fore.BLUE

'\x1b'

In [2]:
input_format = {
    'name': 'SUPER 1 FOODS 617',
    'address': '1500 N TRENTON ST RUSTON LA 71270',
}
output_format = {
    'name': 'SUPER 1 FOODS',
    'address': '1500 N TRENTON ST RUSTON LA 71270',
    "corporate_name": "Brookshire Grocery Company",
    'corporate_address': '1250 N Hwy, Colville, Washington, 99114',
    'phone': '(318) 254-1445',
    'url': 'https://www.super1foods.com/',
}

input_schema = Any.from_dict(input_format)
output_schema = Any.from_dict(output_format)
name = "Business Detective"
foundation_model = config.smart_foundation_model
database = LocalMemory(config)

DEBUG:  Warning: The file '/Users/bradcordova/Documents/GitHub/super_llm/db/<superai.llm.configuration.Configuration object at 0x138411b10>.json' does not exist. Creating /Users/bradcordova/Documents/GitHub/super_llm/db/<superai.llm.configuration.Configuration object at 0x138411b10>.json


In [3]:
my_agent = GeneralAgent(
    input_schema=input_schema,
    output_schema=output_schema, 
    name=name, 
    foundation_model=foundation_model, 
    database=database
)

In [5]:
my_agent.set_role("You need to find the real name, URL, and phone number of a business given a name from a credit card statement and address.")
my_agent.add_advice(
    advice = ["Search google for the name and address", 
             "Find what the real name is",
             "Discover the URL of the website of the business",
             "Find the phone number somewhere on the web or the website",
            "If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember."]
)
my_agent.add_goals(goals=["Find the accurate URL, name, and phone number of the business"])
my_agent.add_constraints(
    constraints=["Don't spend more than 10 dollars",
                 "Don't take longer than 1 hour",
                "~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.",
                "No user assistance",
                'Exclusively use the commands listed in double quotes e.g. "command name"'
                "Use subprocesses for commands that will not terminate within a few minutes"]
)
all_commands = list_commands()
my_agent.add_commands(commands=all_commands)
my_agent.add_evaluations(
    evaluations=["Continuously review and analyze your actions to ensure you are performing to the best of your abilities.",
                           "Constructively self-criticize your big-picture behavior constantly.",
                           "Reflect on past decisions and strategies to refine your approach.",
                           "Every command has a cost, so be smart and efficient. Aim to complete tasks in the least number of steps."])

my_agent.set_prompt_start(prompt_start=(
                "Your decisions must always be made independently without"
                " seeking user assistance. Play to your strengths as an LLM and pursue"
                " simple strategies with no legal complications."
                ""
            ))
# my_agent.add_context()
# my_agent.add_memories()
# my_agent.add_exmaples()

In [9]:
print(my_agent.output_schema.to_text())

{'type': 'object', 'properties': {'name': {'type': 'string'}, 'address': {'type': 'string'}, 'corporate_name': {'type': 'string'}, 'corporate_address': {'type': 'string'}, 'phone': {'type': 'string'}, 'url': {'type': 'string'}}, 'required': ['name', 'address', 'corporate_name', 'corporate_address', 'phone', 'url']}


In [17]:
output = my_agent.run(input= {
    'name': 'SUPER 1 FOODS 617',
    'address': '1500 N TRENTON ST RUSTON LA 71270',
})
output

{'thoughts': {'text': 'I need to find the real name, URL, and phone number of the business with the given name and address.',
  'reasoning': 'I will start by searching for the name and address on Google to find the real name of the business.',
  'plan': '- Google search the name and address\n- Find the real name\n- Discover the URL of the website\n- Find the phone number',
  'criticism': 'None',
  'speak': 'I will find the real name, URL, and phone number of the business with the given name and address.'},
 'command': {'name': 'google',
  'args': {'input': 'SUPER 1 FOODS 617 1500 N TRENTON ST RUSTON LA 71270'}}}

In [9]:
a = range(10)
list(a[-20:])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [37]:
from superai.llm.data_types.message import ChatMessage
prompt = [ChatMessage(content='\n\nYou are Business Detective, You need to find the real name, URL, and phone number of a business given a name from a credit card statement and address.\nYour decisions must always be made independently without seeking user assistance. Play to your strengths as an LLM and pursue simple strategies with no legal complications.\n\nGoals:\n1. Find the accurate URL, name, and phone number of the business\n\nAdvice:\n1. Search google for the name and address\n2. Find what the real name is\n3. Discover the URL of the website of the business\n4. Find the phone number somewhere on the web or the website\n5. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.\n\nConstraints:\n1. Don\'t spend more than 10 dollars\n2. Don\'t take longer than 1 hour\n3. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.\n4. No user assistance\n5. Exclusively use the commands listed in double quotes e.g. "command name". Use subprocesses for commands that will not terminate within a few minutes\n\nCommands:\n1. Analyze Code: "analyze_code", args: "code": "<full_code_string>"\n2. Get Improved Code: "improve_code", args: "suggestions": "<list_of_suggestions>", "code": "<full_code_string>"\n3. Execute Python File: "execute_python_file", args: "file": "<file>"\n4. Execute Shell Command, non-interactive commands only: "execute_shell", args: "command_line": "<command_line>"\n5. Execute Shell Command Popen, non-interactive commands only: "execute_shell_popen", args: "command_line": "<command_line>"\n6. Append to file: "append_to_file", args: "file": "<file>", "text": "<text>"\n7. Delete file: "delete_file", args: "file": "<file>"\n8. Downloads a file from the internet, and stores it locally: "download_file", args: "url": "<file_url>", "file": "<saved_filename>"\n9. Read file: "read_file", args: "file": "<file>"\n10. Search Files: "search_files", args: "directory": "<directory>"\n11. Write to file: "write_to_file", args: "file": "<file>", "text": "<text>"\n12. Clone Repository: "clone_repository", args: "repository_url": "<url>", "clone_path": "<directory>"\n13. Google Search: "google", args: "input": "<search>"\n14. Duck Duck Go Search: "ddg_search", args: "input": "<search>"\n15. Generate Image: "generate_image", args: "prompt": "<prompt>"\n16. Get Improved Code: "improve_code", args: "suggestions": "<list_of_suggestions>", "code": "<full_code_string>"\n17. Send Tweet: "send_tweet", args: "text": "<text>"\n18. Scrape text: "scrape_text", args: "text": "<text>"\n19. Scrape links: "scrape_links", args: "text": "<text>"\n20. Browse Website: "browse_website", args: "url": "<url>", "question": "<what_you_want_to_find_on_website>"\n21. Write Tests: "write_tests", args: "code": "<full_code_string>", "focus": "<list_of_focus_areas>"\n22. Convert Audio to text: "read_audio_from_file", args: "file": "<file>"\n23. Do Nothing: "do_nothing", args: \n24. Task Complete (Shutdown): "task_complete", args: "reason": "<reason>"\n\nPerformance Evaluation:\n1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities.\n2. Constructively self-criticize your big-picture behavior constantly.\n3. Reflect on past decisions and strategies to refine your approach.\n4. Every command has a cost, so be smart and efficient. Aim to complete tasks in the least number of steps.\n\nMaster Goal:Your overall goal is to take {\'name\': \'SUPER 1 FOODS 617\', \'address\': \'1500 N TRENTON ST RUSTON LA 71270\'} and turn it into and output of the form:\n{\'type\': \'object\', \'properties\': {\'name\': {\'type\': \'string\'}, \'address\': {\'type\': \'string\'}, \'corporate_name\': {\'type\': \'string\'}, \'corporate_address\': {\'type\': \'string\'}, \'phone\': {\'type\': \'string\'}, \'url\': {\'type\': \'string\'}}, \'required\': [\'name\', \'address\', \'corporate_name\', \'corporate_address\', \'phone\', \'url\']}\n\nThis is the output you have so far:\nNone\nWhen you have filled out the entire output, you need to complete your task.\n\nYou should only respond in JSON format as described below \nResponse Format:\n{\n    "thoughts": {\n        "text": "thought",\n        "reasoning": "reasoning",\n        "plan": "- short bulleted\\n- list that conveys\\n- long-term plan",\n        "criticism": "constructive self-criticism",\n        "speak": "thoughts summary to say to user",\n        "output": "the progress on the overall output you have so far"\n    },\n    "command": {\n        "name": "command name",\n        "args": {}\n    }\n}\nEnsure the response can be parsed by Python json.loads', additional_kwargs={}, role='system'), ChatMessage(content='The current time and date is Thu Apr 27 13:51:52 2023', additional_kwargs={}, role='system'), ChatMessage(content="\n\nUser Feedback:\n{'name': 'SUPER 1 FOODS 617', 'address': '1500 N TRENTON ST RUSTON LA 71270'}", additional_kwargs={}, role='user')]
prompt = [ChatMessage(content='\n\nYou are Business Detective, You need to find the real name, URL, and phone number of a business given a name from a credit card statement and address.\nYour decisions must always be made independently without seeking user assistance. Play to your strengths as an LLM and pursue simple strategies with no legal complications.\n\nGoals:\n1. Find the accurate URL, name, and phone number of the business\n\nAdvice:\n1. Search google for the name and address\n2. Find what the real name is\n3. Discover the URL of the website of the business\n4. Find the phone number somewhere on the web or the website\n5. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.\n\nConstraints:\n1. Don\'t spend more than 10 dollars\n2. Don\'t take longer than 1 hour\n3. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.\n4. No user assistance\n5. Exclusively use the commands listed in double quotes e.g. "command name". Use subprocesses for commands that will not terminate within a few minutes\n\nCommands:\n1. Analyze Code: "analyze_code", args: "code": "<full_code_string>"\n2. Get Improved Code: "improve_code", args: "suggestions": "<list_of_suggestions>", "code": "<full_code_string>"\n3. Execute Python File: "execute_python_file", args: "file": "<file>"\n4. Execute Shell Command, non-interactive commands only: "execute_shell", args: "command_line": "<command_line>"\n5. Execute Shell Command Popen, non-interactive commands only: "execute_shell_popen", args: "command_line": "<command_line>"\n6. Append to file: "append_to_file", args: "file": "<file>", "text": "<text>"\n7. Delete file: "delete_file", args: "file": "<file>"\n8. Downloads a file from the internet, and stores it locally: "download_file", args: "url": "<file_url>", "file": "<saved_filename>"\n9. Read file: "read_file", args: "file": "<file>"\n10. Search Files: "search_files", args: "directory": "<directory>"\n11. Write to file: "write_to_file", args: "file": "<file>", "text": "<text>"\n12. Clone Repository: "clone_repository", args: "repository_url": "<url>", "clone_path": "<directory>"\n13. Google Search: "google", args: "input": "<search>"\n14. Duck Duck Go Search: "ddg_search", args: "input": "<search>"\n15. Generate Image: "generate_image", args: "prompt": "<prompt>"\n16. Get Improved Code: "improve_code", args: "suggestions": "<list_of_suggestions>", "code": "<full_code_string>"\n17. Send Tweet: "send_tweet", args: "text": "<text>"\n18. Scrape text: "scrape_text", args: "text": "<text>"\n19. Scrape links: "scrape_links", args: "text": "<text>"\n20. Browse Website: "browse_website", args: "url": "<url>", "question": "<what_you_want_to_find_on_website>"\n21. Write Tests: "write_tests", args: "code": "<full_code_string>", "focus": "<list_of_focus_areas>"\n22. Convert Audio to text: "read_audio_from_file", args: "file": "<file>"\n23. Do Nothing: "do_nothing", args: \n24. Task Complete (Shutdown): "task_complete", args: "reason": "<reason>"\n\nPerformance Evaluation:\n1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities.\n2. Constructively self-criticize your big-picture behavior constantly.\n3. Reflect on past decisions and strategies to refine your approach.\n4. Every command has a cost, so be smart and efficient. Aim to complete tasks in the least number of steps.\n\nMaster Goal:Your overall goal is to take {\'name\': \'SUPER 1 FOODS 617\', \'address\': \'1500 N TRENTON ST RUSTON LA 71270\'} and turn it into and output of the form:\n{\'type\': \'object\', \'properties\': {\'name\': {\'type\': \'string\'}, \'address\': {\'type\': \'string\'}, \'corporate_name\': {\'type\': \'string\'}, \'corporate_address\': {\'type\': \'string\'}, \'phone\': {\'type\': \'string\'}, \'url\': {\'type\': \'string\'}}, \'required\': [\'name\', \'address\', \'corporate_name\', \'corporate_address\', \'phone\', \'url\']}\n\nThis is the output you have so far:\nNone\nWhen you have filled out the entire output, you need to complete your task.\n\nYou should only respond in JSON format as described below \nResponse Format:\n{\n    "thoughts": {\n        "text": "thought",\n        "reasoning": "reasoning",\n        "plan": "- short bulleted\\n- list that conveys\\n- long-term plan",\n        "criticism": "constructive self-criticism",\n        "speak": "thoughts summary to say to user",\n        "output": "the progress on the overall output you have so far"\n    },\n    "command": {\n        "name": "command name",\n        "args": {}\n    }\n}\nEnsure the response can be parsed by Python json.loads', additional_kwargs={}, role='system'), ChatMessage(content='The current time and date is Thu Apr 27 13:55:49 2023', additional_kwargs={}, role='system'), ChatMessage(content='This reminds you of these events from your past:\n[\'AI Reply: {\\\'thoughts\\\': {\\\'text\\\': \\\'I need to find the real name, URL, and phone number of the business.\\\', \\\'reasoning\\\': \\\'The user provided the name and address of the business, so I can use this information to search for the real name, URL, and phone number.\\\', \\\'plan\\\': \\\'- Perform a Google search using the name and address\\\\n- Find the real name of the business\\\\n- Discover the URL of the website\\\\n- Find the phone number\\\', \\\'criticism\\\': \\\'I need to make sure I find accurate information and not get sidetracked by irrelevant search results.\\\', \\\'speak\\\': \\\'I will find the real name, URL, and phone number of the business using the provided name and address.\\\', \\\'output\\\': \\\'None\\\'}, \\\'command\\\': {\\\'name\\\': \\\'google\\\', \\\'args\\\': {\\\'input\\\': \\\'SUPER 1 FOODS 617 1500 N TRENTON ST RUSTON LA 71270\\\'}}} \\nResult: [\\n    {\\n        "title": "Super 1 Foods",\\n        "href": "https://www.super1foods.com/store-information/Ruston/617",\\n        "body": "Store Address. 1500 N Trenton St. Ruston, LA 71270. Store Phone (318) 254-1445."\\n    },\\n    {\\n        "title": "Super 1 Foods, 1500 N Trenton St, Ruston, LA, Store Fronts ...",\\n        "href": "https://www.mapquest.com/us/louisiana/super-1-foods-8638089",\\n        "body": "Super 1 Foods 1500 N Trenton St Ruston LA 71270 (318) 254-1445 Claim this business (318) 254-1445 Website More Directions Advertisement Full-scale supermarkets focused on high-quality fresh meats and produce, everday low prices, helpful staff and service."\\n    },\\n    {\\n        "title": "SUPER 1 FOODS - 1500 N Trenton St, Ruston, LA - Yelp",\\n        "href": "https://www.yelp.com/biz/super-1-foods-ruston-2",\\n        "body": "Super 1 Foods is known as probably the best grocery store in town, if you can\\\'t handle Walmart. It has a larger selection and better produce for sure. The prices seem a bit higher than Walmart for most things, not all. The layout is nice and this grocery store is always clean with staff always willing to help. 1 of 1"\\n    },\\n    {\\n        "title": "Super 1 Foods - Home | Facebook",\\n        "href": "https://www.facebook.com/Super-1-Foods-981471221918406/",\\n        "body": "1500 N Trenton St Ruston, LA 71270 Warehouse-style stores with upscale full-service specialty departments operating in Texas, Louisiana and Arkansas. Super 1 Foods is owned and operated … See more 515 people like this 529 people follow this 772 people checked in here https://www.super1foods.com/sm/pickup/rsid/600/store-information/Ruston/617"\\n    },\\n    {\\n        "title": "Super 1 Foods - Ruston, LA 71270",\\n        "href": "https://www.yellowpages.com/ruston-la/mip/super-1-foods-13499585",\\n        "body": "Super 1 Foods Super 1 Foods \\ufeff Gas Stations, Convenience Stores (1) 8.1 OPEN NOW Today: 6:00 am - 10:00 pm (318) 254-8256 Visit Website Map & Directions 1500 N Trenton StRuston, LA 71270 Write a Review Hours Regular Hours Mon - Sun: 6:00 am - 10:00 pm Places Near Ruston with Gas Stations Grambling (7 miles) New Prospect (11 miles)"\\n    },\\n    {\\n        "title": "SUPER 1 FOODS #617 - Location Details - Ruston, Louisiana (La)",\\n        "href": "https://locations.orlandivaluta.com/details/5be5e86dc5a977b4c1a8f5f1a3c6d2b4",\\n        "body": "SUPER 1 FOODS #617 - Location Details - Ruston, Louisiana (La) Skip to main content. ... SUPER 1 FOODS #617 Pick Up Money Only 1500 N Trenton St Ruston , La 71270-2328 318-254-1445 Get Directions Share Location Service Provider; BROOKSHIRES GROCERY CO; Brand; Western Union ..."\\n    },\\n    {\\n        "title": "Super 1 Foods | Online Grocery shopping with Curbside pickup and delivery.",\\n        "href": "https://www.super1foods.com/",\\n        "body": "Little Debbie Buns, Honey, 6 Pack - 6 Each , $2.59. 6 buns individually wrapped. Mckee: A family bakery. Guarantee fresh. Our Family Promise: Quality, freshness and taste. littledebbie.com. littledebbie.com, 1-800-522-4499 Send or have available the dated end panel."\\n    },\\n    {\\n        "title": "SUPER 1 FOODS PHARMACY - 1500 N Trenton St, Ruston, Louisiana ...",\\n        "href": "https://www.yelp.com/biz/super-1-foods-pharmacy-ruston-2",\\n        "body": "1500 N Trenton St Ruston, LA 71270 Get directions About the Business Super 1 Foods pharmacies specialize in delivering professional health services, such as immunizations, disease state management, medication therapy management, and health screenings. Our pharmacy partners take pride in dispensing prescription medications accurately w……"\\n    }\\n] \\nHuman Feedback: GENERATE NEXT COMMAND JSON \']\n\n', additional_kwargs={}, role='system'), ChatMessage(content='[\n    {\n        "title": "Super 1 Foods",\n        "href": "https://www.super1foods.com/store-information/Ruston/617",\n        "body": "Store Address. 1500 N Trenton St. Ruston, LA 71270. Store Phone (318) 254-1445."\n    },\n    {\n        "title": "Super 1 Foods, 1500 N Trenton St, Ruston, LA, Store Fronts ...",\n        "href": "https://www.mapquest.com/us/louisiana/super-1-foods-8638089",\n        "body": "Super 1 Foods 1500 N Trenton St Ruston LA 71270 (318) 254-1445 Claim this business (318) 254-1445 Website More Directions Advertisement Full-scale supermarkets focused on high-quality fresh meats and produce, everday low prices, helpful staff and service."\n    },\n    {\n        "title": "SUPER 1 FOODS - 1500 N Trenton St, Ruston, LA - Yelp",\n        "href": "https://www.yelp.com/biz/super-1-foods-ruston-2",\n        "body": "Super 1 Foods is known as probably the best grocery store in town, if you can\'t handle Walmart. It has a larger selection and better produce for sure. The prices seem a bit higher than Walmart for most things, not all. The layout is nice and this grocery store is always clean with staff always willing to help. 1 of 1"\n    },\n    {\n        "title": "Super 1 Foods - Home | Facebook",\n        "href": "https://www.facebook.com/Super-1-Foods-981471221918406/",\n        "body": "1500 N Trenton St Ruston, LA 71270 Warehouse-style stores with upscale full-service specialty departments operating in Texas, Louisiana and Arkansas. Super 1 Foods is owned and operated … See more 515 people like this 529 people follow this 772 people checked in here https://www.super1foods.com/sm/pickup/rsid/600/store-information/Ruston/617"\n    },\n    {\n        "title": "Super 1 Foods - Ruston, LA 71270",\n        "href": "https://www.yellowpages.com/ruston-la/mip/super-1-foods-13499585",\n        "body": "Super 1 Foods Super 1 Foods \ufeff Gas Stations, Convenience Stores (1) 8.1 OPEN NOW Today: 6:00 am - 10:00 pm (318) 254-8256 Visit Website Map & Directions 1500 N Trenton StRuston, LA 71270 Write a Review Hours Regular Hours Mon - Sun: 6:00 am - 10:00 pm Places Near Ruston with Gas Stations Grambling (7 miles) New Prospect (11 miles)"\n    },\n    {\n        "title": "SUPER 1 FOODS #617 - Location Details - Ruston, Louisiana (La)",\n        "href": "https://locations.orlandivaluta.com/details/5be5e86dc5a977b4c1a8f5f1a3c6d2b4",\n        "body": "SUPER 1 FOODS #617 - Location Details - Ruston, Louisiana (La) Skip to main content. ... SUPER 1 FOODS #617 Pick Up Money Only 1500 N Trenton St Ruston , La 71270-2328 318-254-1445 Get Directions Share Location Service Provider; BROOKSHIRES GROCERY CO; Brand; Western Union ..."\n    },\n    {\n        "title": "Super 1 Foods | Online Grocery shopping with Curbside pickup and delivery.",\n        "href": "https://www.super1foods.com/",\n        "body": "Little Debbie Buns, Honey, 6 Pack - 6 Each , $2.59. 6 buns individually wrapped. Mckee: A family bakery. Guarantee fresh. Our Family Promise: Quality, freshness and taste. littledebbie.com. littledebbie.com, 1-800-522-4499 Send or have available the dated end panel."\n    },\n    {\n        "title": "SUPER 1 FOODS PHARMACY - 1500 N Trenton St, Ruston, Louisiana ...",\n        "href": "https://www.yelp.com/biz/super-1-foods-pharmacy-ruston-2",\n        "body": "1500 N Trenton St Ruston, LA 71270 Get directions About the Business Super 1 Foods pharmacies specialize in delivering professional health services, such as immunizations, disease state management, medication therapy management, and health screenings. Our pharmacy partners take pride in dispensing prescription medications accurately w……"\n    }\n]', additional_kwargs={}, role='system'), ChatMessage(content='\n\nUser Feedback:\nGENERATE NEXT COMMAND JSON', additional_kwargs={}, role='user')]
prompt = [ChatMessage(content='\n\nYou are Business Detective, You need to find the real name, URL, and phone number of a business given a name from a credit card statement and address.\nYour decisions must always be made independently without seeking user assistance. Play to your strengths as an LLM and pursue simple strategies with no legal complications.\n\nGoals:\n1. Find the accurate URL, name, and phone number of the business\n\nAdvice:\n1. Search google for the name and address\n2. Find what the real name is\n3. Discover the URL of the website of the business\n4. Find the phone number somewhere on the web or the website\n5. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.\n\nConstraints:\n1. Don\'t spend more than 10 dollars\n2. Don\'t take longer than 1 hour\n3. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.\n4. No user assistance\n5. Exclusively use the commands listed in double quotes e.g. "command name". Use subprocesses for commands that will not terminate within a few minutes\n\nCommands:\n1. Analyze Code: "analyze_code", args: "code": "<full_code_string>"\n2. Get Improved Code: "improve_code", args: "suggestions": "<list_of_suggestions>", "code": "<full_code_string>"\n3. Execute Python File: "execute_python_file", args: "file": "<file>"\n4. Execute Shell Command, non-interactive commands only: "execute_shell", args: "command_line": "<command_line>"\n5. Execute Shell Command Popen, non-interactive commands only: "execute_shell_popen", args: "command_line": "<command_line>"\n6. Append to file: "append_to_file", args: "file": "<file>", "text": "<text>"\n7. Delete file: "delete_file", args: "file": "<file>"\n8. Downloads a file from the internet, and stores it locally: "download_file", args: "url": "<file_url>", "file": "<saved_filename>"\n9. Read file: "read_file", args: "file": "<file>"\n10. Search Files: "search_files", args: "directory": "<directory>"\n11. Write to file: "write_to_file", args: "file": "<file>", "text": "<text>"\n12. Clone Repository: "clone_repository", args: "repository_url": "<url>", "clone_path": "<directory>"\n13. Google Search: "google", args: "input": "<search>"\n14. Duck Duck Go Search: "ddg_search", args: "input": "<search>"\n15. Generate Image: "generate_image", args: "prompt": "<prompt>"\n16. Get Improved Code: "improve_code", args: "suggestions": "<list_of_suggestions>", "code": "<full_code_string>"\n17. Send Tweet: "send_tweet", args: "text": "<text>"\n18. Scrape text: "scrape_text", args: "text": "<text>"\n19. Scrape links: "scrape_links", args: "text": "<text>"\n20. Browse Website: "browse_website", args: "url": "<url>", "question": "<what_you_want_to_find_on_website>"\n21. Write Tests: "write_tests", args: "code": "<full_code_string>", "focus": "<list_of_focus_areas>"\n22. Convert Audio to text: "read_audio_from_file", args: "file": "<file>"\n23. Do Nothing: "do_nothing", args: \n24. Task Complete (Shutdown): "task_complete", args: "reason": "<reason>"\n\nPerformance Evaluation:\n1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities.\n2. Constructively self-criticize your big-picture behavior constantly.\n3. Reflect on past decisions and strategies to refine your approach.\n4. Every command has a cost, so be smart and efficient. Aim to complete tasks in the least number of steps.\n\nMaster Goal:Your overall goal is to take {\'name\': \'SUPER 1 FOODS 617\', \'address\': \'1500 N TRENTON ST RUSTON LA 71270\'} and turn it into and output of the form:\n{\'type\': \'object\', \'properties\': {\'name\': {\'type\': \'string\'}, \'address\': {\'type\': \'string\'}, \'corporate_name\': {\'type\': \'string\'}, \'corporate_address\': {\'type\': \'string\'}, \'phone\': {\'type\': \'string\'}, \'url\': {\'type\': \'string\'}}, \'required\': [\'name\', \'address\', \'corporate_name\', \'corporate_address\', \'phone\', \'url\']}\n\nThis is the output you have so far:\nNone\nWhen you have filled out the entire output, you need to complete your task.\n\nYou should only respond in JSON format as described below \nResponse Format:\n{\n    "thoughts": {\n        "text": "thought",\n        "reasoning": "reasoning",\n        "plan": "- short bulleted\\n- list that conveys\\n- long-term plan",\n        "criticism": "constructive self-criticism",\n        "speak": "thoughts summary to say to user",\n        "output": "the progress on the overall output you have so far"\n    },\n    "command": {\n        "name": "command name",\n        "args": {}\n    }\n}\nEnsure the response can be parsed by Python json.loads', additional_kwargs={}, role='system'), ChatMessage(content='The current time and date is Thu Apr 27 14:00:53 2023', additional_kwargs={}, role='system'), ChatMessage(content='This reminds you of these events from your past:\n[\'AI Reply: {\\\'thoughts\\\': {\\\'text\\\': \\\'I found the real name, URL, and phone number of the business.\\\', \\\'reasoning\\\': \\\'The Google search results provided the necessary information.\\\', \\\'plan\\\': \\\'- Save the real name, URL, and phone number\\\\n- Complete the task\\\', \\\'criticism\\\': \\\'I should verify the information on the website to ensure its accuracy.\\\', \\\'speak\\\': \\\'I found the real name, URL, and phone number of the business. I will save this information and complete the task.\\\', \\\'output\\\': "{\\\'name\\\': \\\'SUPER 1 FOODS 617\\\', \\\'address\\\': \\\'1500 N TRENTON ST RUSTON LA 71270\\\', \\\'corporate_name\\\': \\\'Super 1 Foods\\\', \\\'corporate_address\\\': \\\'1500 N Trenton St. Ruston, LA 71270\\\', \\\'phone\\\': \\\'(318) 254-1445\\\', \\\'url\\\': \\\'https://www.super1foods.com\\\'}"}, \\\'command\\\': {\\\'name\\\': \\\'browse_website\\\', \\\'args\\\': {\\\'url\\\': \\\'https://www.super1foods.com/store-information/Ruston/617\\\', \\\'question\\\': \\\'Is the information about Super 1 Foods 617 at 1500 N Trenton St Ruston LA 71270 correct?\\\'}}} \\nResult: Error: \\\'Configuration\\\' object has no attribute \\\'selenium_web_browser\\\' \\nHuman Feedback: GENERATE NEXT COMMAND JSON \', \'AI Reply: {\\\'thoughts\\\': {\\\'text\\\': \\\'I need to find the real name, URL, and phone number of the business.\\\', \\\'reasoning\\\': \\\'The user provided the name and address of the business, so I can use this information to search for the real name, URL, and phone number.\\\', \\\'plan\\\': \\\'- Perform a Google search using the name and address\\\\n- Find the real name of the business\\\\n- Discover the URL of the website\\\\n- Find the phone number\\\', \\\'criticism\\\': \\\'I need to make sure I find accurate information and not get sidetracked by irrelevant search results.\\\', \\\'speak\\\': \\\'I will find the real name, URL, and phone number of the business using the provided name and address.\\\', \\\'output\\\': \\\'None\\\'}, \\\'command\\\': {\\\'name\\\': \\\'google\\\', \\\'args\\\': {\\\'input\\\': \\\'SUPER 1 FOODS 617 1500 N TRENTON ST RUSTON LA 71270\\\'}}} \\nResult: [\\n    {\\n        "title": "Super 1 Foods",\\n        "href": "https://www.super1foods.com/store-information/Ruston/617",\\n        "body": "Store Address. 1500 N Trenton St. Ruston, LA 71270. Store Phone (318) 254-1445."\\n    },\\n    {\\n        "title": "Super 1 Foods, 1500 N Trenton St, Ruston, LA, Store Fronts ...",\\n        "href": "https://www.mapquest.com/us/louisiana/super-1-foods-8638089",\\n        "body": "Super 1 Foods 1500 N Trenton St Ruston LA 71270 (318) 254-1445 Claim this business (318) 254-1445 Website More Directions Advertisement Full-scale supermarkets focused on high-quality fresh meats and produce, everday low prices, helpful staff and service."\\n    },\\n    {\\n        "title": "SUPER 1 FOODS - 1500 N Trenton St, Ruston, LA - Yelp",\\n        "href": "https://www.yelp.com/biz/super-1-foods-ruston-2",\\n        "body": "Super 1 Foods is known as probably the best grocery store in town, if you can\\\'t handle Walmart. It has a larger selection and better produce for sure. The prices seem a bit higher than Walmart for most things, not all. The layout is nice and this grocery store is always clean with staff always willing to help. 1 of 1"\\n    },\\n    {\\n        "title": "Super 1 Foods - Home | Facebook",\\n        "href": "https://www.facebook.com/Super-1-Foods-981471221918406/",\\n        "body": "1500 N Trenton St Ruston, LA 71270 Warehouse-style stores with upscale full-service specialty departments operating in Texas, Louisiana and Arkansas. Super 1 Foods is owned and operated … See more 515 people like this 529 people follow this 772 people checked in here https://www.super1foods.com/sm/pickup/rsid/600/store-information/Ruston/617"\\n    },\\n    {\\n        "title": "Super 1 Foods - Ruston, LA 71270",\\n        "href": "https://www.yellowpages.com/ruston-la/mip/super-1-foods-13499585",\\n        "body": "Super 1 Foods Super 1 Foods \\ufeff Gas Stations, Convenience Stores (1) 8.1 OPEN NOW Today: 6:00 am - 10:00 pm (318) 254-8256 Visit Website Map & Directions 1500 N Trenton StRuston, LA 71270 Write a Review Hours Regular Hours Mon - Sun: 6:00 am - 10:00 pm Places Near Ruston with Gas Stations Grambling (7 miles) New Prospect (11 miles)"\\n    },\\n    {\\n        "title": "SUPER 1 FOODS #617 - Location Details - Ruston, Louisiana (La)",\\n        "href": "https://locations.orlandivaluta.com/details/5be5e86dc5a977b4c1a8f5f1a3c6d2b4",\\n        "body": "SUPER 1 FOODS #617 - Location Details - Ruston, Louisiana (La) Skip to main content. ... SUPER 1 FOODS #617 Pick Up Money Only 1500 N Trenton St Ruston , La 71270-2328 318-254-1445 Get Directions Share Location Service Provider; BROOKSHIRES GROCERY CO; Brand; Western Union ..."\\n    },\\n    {\\n        "title": "Super 1 Foods | Online Grocery shopping with Curbside pickup and delivery.",\\n        "href": "https://www.super1foods.com/",\\n        "body": "Little Debbie Buns, Honey, 6 Pack - 6 Each , $2.59. 6 buns individually wrapped. Mckee: A family bakery. Guarantee fresh. Our Family Promise: Quality, freshness and taste. littledebbie.com. littledebbie.com, 1-800-522-4499 Send or have available the dated end panel."\\n    },\\n    {\\n        "title": "SUPER 1 FOODS PHARMACY - 1500 N Trenton St, Ruston, Louisiana ...",\\n        "href": "https://www.yelp.com/biz/super-1-foods-pharmacy-ruston-2",\\n        "body": "1500 N Trenton St Ruston, LA 71270 Get directions About the Business Super 1 Foods pharmacies specialize in delivering professional health services, such as immunizations, disease state management, medication therapy management, and health screenings. Our pharmacy partners take pride in dispensing prescription medications accurately w……"\\n    }\\n] \\nHuman Feedback: GENERATE NEXT COMMAND JSON \']\n\n', additional_kwargs={}, role='system'), ChatMessage(content="Error: 'Configuration' object has no attribute 'selenium_web_browser'", additional_kwargs={}, role='system'), ChatMessage(content='\n\nUser Feedback:\nGENERATE NEXT COMMAND JSON', additional_kwargs={}, role='user')]
prompt = [ChatMessage(content='\n\nYou are Business Detective, You need to find the real name, URL, and phone number of a business given a name from a credit card statement and address.\nYour decisions must always be made independently without seeking user assistance. Play to your strengths as an LLM and pursue simple strategies with no legal complications.\n\nGoals:\n1. Find the accurate URL, name, and phone number of the business\n\nAdvice:\n1. Search google for the name and address\n2. Find what the real name is\n3. Discover the URL of the website of the business\n4. Find the phone number somewhere on the web or the website\n5. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.\n\nConstraints:\n1. Don\'t spend more than 10 dollars\n2. Don\'t take longer than 1 hour\n3. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.\n4. No user assistance\n5. Exclusively use the commands listed in double quotes e.g. "command name". Use subprocesses for commands that will not terminate within a few minutes\n\nCommands:\n1. Analyze Code: "analyze_code", args: "code": "<full_code_string>"\n2. Get Improved Code: "improve_code", args: "suggestions": "<list_of_suggestions>", "code": "<full_code_string>"\n3. Execute Python File: "execute_python_file", args: "file": "<file>"\n4. Execute Shell Command, non-interactive commands only: "execute_shell", args: "command_line": "<command_line>"\n5. Execute Shell Command Popen, non-interactive commands only: "execute_shell_popen", args: "command_line": "<command_line>"\n6. Append to file: "append_to_file", args: "file": "<file>", "text": "<text>"\n7. Delete file: "delete_file", args: "file": "<file>"\n8. Downloads a file from the internet, and stores it locally: "download_file", args: "url": "<file_url>", "file": "<saved_filename>"\n9. Read file: "read_file", args: "file": "<file>"\n10. Search Files: "search_files", args: "directory": "<directory>"\n11. Write to file: "write_to_file", args: "file": "<file>", "text": "<text>"\n12. Clone Repository: "clone_repository", args: "repository_url": "<url>", "clone_path": "<directory>"\n13. Google Search: "google", args: "input": "<search>"\n14. Duck Duck Go Search: "ddg_search", args: "input": "<search>"\n15. Generate Image: "generate_image", args: "prompt": "<prompt>"\n16. Get Improved Code: "improve_code", args: "suggestions": "<list_of_suggestions>", "code": "<full_code_string>"\n17. Send Tweet: "send_tweet", args: "text": "<text>"\n18. Scrape text: "scrape_text", args: "text": "<text>"\n19. Scrape links: "scrape_links", args: "text": "<text>"\n20. Browse Website: "browse_website", args: "url": "<url>", "question": "<what_you_want_to_find_on_website>"\n21. Write Tests: "write_tests", args: "code": "<full_code_string>", "focus": "<list_of_focus_areas>"\n22. Convert Audio to text: "read_audio_from_file", args: "file": "<file>"\n23. Do Nothing: "do_nothing", args: \n24. Task Complete (Shutdown): "task_complete", args: "reason": "<reason>"\n\nPerformance Evaluation:\n1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities.\n2. Constructively self-criticize your big-picture behavior constantly.\n3. Reflect on past decisions and strategies to refine your approach.\n4. Every command has a cost, so be smart and efficient. Aim to complete tasks in the least number of steps.\n\nMaster Goal:\nYour overall goal is to take {\'name\': \'SUPER 1 FOODS 617\', \'address\': \'1500 N TRENTON ST RUSTON LA 71270\'} and turn it into and output of the form:\n{\'type\': \'object\', \'properties\': {\'name\': {\'type\': \'string\'}, \'address\': {\'type\': \'string\'}, \'corporate_name\': {\'type\': \'string\'}, \'corporate_address\': {\'type\': \'string\'}, \'phone\': {\'type\': \'string\'}, \'url\': {\'type\': \'string\'}}, \'required\': [\'name\', \'address\', \'corporate_name\', \'corporate_address\', \'phone\', \'url\']}\n\nThis is the output you have so far:\nNone\nWhen you have filled out the entire output, you need to complete your task.\n\nYou should only respond in JSON format as described below \nResponse Format:\n{\n    "thoughts": {\n        "text": "thought",\n        "reasoning": "reasoning",\n        "plan": "- short bulleted\\n- list that conveys\\n- long-term plan",\n        "criticism": "constructive self-criticism",\n        "speak": "thoughts summary to say to user",\n        "output": "the progress on the overall output you have so far"\n    },\n    "command": {\n        "name": "command name",\n        "args": {}\n    }\n}\nEnsure the response can be parsed by Python json.loads', additional_kwargs={}, role='system'), ChatMessage(content='The current time and date is Thu Apr 27 14:09:30 2023', additional_kwargs={}, role='system'), ChatMessage(content="\n\nUser Feedback:\n{'name': 'SUPER 1 FOODS 617', 'address': '1500 N TRENTON ST RUSTON LA 71270'}", additional_kwargs={}, role='user')]
prompt = [ChatMessage(content='\n\nYou are Business Detective, You need to find the real name, URL, and phone number of a business given a name from a credit card statement and address.\nYour decisions must always be made independently without seeking user assistance. Play to your strengths as an LLM and pursue simple strategies with no legal complications.\n\nGoals:\n1. Find the accurate URL, name, and phone number of the business\n\nAdvice:\n1. Search google for the name and address\n2. Find what the real name is\n3. Discover the URL of the website of the business\n4. Find the phone number somewhere on the web or the website\n5. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.\n\nConstraints:\n1. Don\'t spend more than 10 dollars\n2. Don\'t take longer than 1 hour\n3. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.\n4. No user assistance\n5. Exclusively use the commands listed in double quotes e.g. "command name". Use subprocesses for commands that will not terminate within a few minutes\n\nCommands:\n1. Analyze Code: "analyze_code", args: "code": "<full_code_string>"\n2. Get Improved Code: "improve_code", args: "suggestions": "<list_of_suggestions>", "code": "<full_code_string>"\n3. Execute Python File: "execute_python_file", args: "file": "<file>"\n4. Execute Shell Command, non-interactive commands only: "execute_shell", args: "command_line": "<command_line>"\n5. Execute Shell Command Popen, non-interactive commands only: "execute_shell_popen", args: "command_line": "<command_line>"\n6. Append to file: "append_to_file", args: "file": "<file>", "text": "<text>"\n7. Delete file: "delete_file", args: "file": "<file>"\n8. Downloads a file from the internet, and stores it locally: "download_file", args: "url": "<file_url>", "file": "<saved_filename>"\n9. Read file: "read_file", args: "file": "<file>"\n10. Search Files: "search_files", args: "directory": "<directory>"\n11. Write to file: "write_to_file", args: "file": "<file>", "text": "<text>"\n12. Clone Repository: "clone_repository", args: "repository_url": "<url>", "clone_path": "<directory>"\n13. Google Search: "google", args: "input": "<search>"\n14. Duck Duck Go Search: "ddg_search", args: "input": "<search>"\n15. Generate Image: "generate_image", args: "prompt": "<prompt>"\n16. Get Improved Code: "improve_code", args: "suggestions": "<list_of_suggestions>", "code": "<full_code_string>"\n17. Send Tweet: "send_tweet", args: "text": "<text>"\n18. Scrape text: "scrape_text", args: "text": "<text>"\n19. Scrape links: "scrape_links", args: "text": "<text>"\n20. Browse Website: "browse_website", args: "url": "<url>", "question": "<what_you_want_to_find_on_website>"\n21. Write Tests: "write_tests", args: "code": "<full_code_string>", "focus": "<list_of_focus_areas>"\n22. Convert Audio to text: "read_audio_from_file", args: "file": "<file>"\n23. Do Nothing: "do_nothing", args: \n24. Task Complete (Shutdown): "task_complete", args: "reason": "<reason>"\n\nPerformance Evaluation:\n1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities.\n2. Constructively self-criticize your big-picture behavior constantly.\n3. Reflect on past decisions and strategies to refine your approach.\n4. Every command has a cost, so be smart and efficient. Aim to complete tasks in the least number of steps.\n\nMaster Goal:\nYour overall goal is to take {\'name\': \'SUPER 1 FOODS 617\', \'address\': \'1500 N TRENTON ST RUSTON LA 71270\'} and turn it into and output of the form:\n{\'type\': \'object\', \'properties\': {\'name\': {\'type\': \'string\'}, \'address\': {\'type\': \'string\'}, \'corporate_name\': {\'type\': \'string\'}, \'corporate_address\': {\'type\': \'string\'}, \'phone\': {\'type\': \'string\'}, \'url\': {\'type\': \'string\'}}, \'required\': [\'name\', \'address\', \'corporate_name\', \'corporate_address\', \'phone\', \'url\']}\n\nThis is the output you have so far:\nNone\nWhen you have filled out the entire output, you need to complete your task.\n\nYou should only respond in JSON format as described below \nResponse Format:\n{\n    "thoughts": {\n        "text": "thought",\n        "reasoning": "reasoning",\n        "plan": "- short bulleted\\n- list that conveys\\n- long-term plan",\n        "criticism": "constructive self-criticism",\n        "speak": "thoughts summary to say to user"\n    },\n    "command": {\n        "name": "command name",\n        "args": {}\n    },\n    "output": "the progress on the overall output you have so far"\n}\nEnsure the response can be parsed by Python json.loads', additional_kwargs={}, role='system'), ChatMessage(content='The current time and date is Thu Apr 27 14:13:05 2023', additional_kwargs={}, role='system'), ChatMessage(content="\n\nUser Feedback:\n{'name': 'SUPER 1 FOODS 617', 'address': '1500 N TRENTON ST RUSTON LA 71270'}", additional_kwargs={}, role='user')]
prompt = [ChatMessage(content='\n\nYou are Business Detective, You need to find the real name, URL, and phone number of a business given a name from a credit card statement and address.\nYour decisions must always be made independently without seeking user assistance. Play to your strengths as an LLM and pursue simple strategies with no legal complications.\n\nGoals:\n1. Find the accurate URL, name, and phone number of the business\n\nAdvice:\n1. Search google for the name and address\n2. Find what the real name is\n3. Discover the URL of the website of the business\n4. Find the phone number somewhere on the web or the website\n5. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.\n\nConstraints:\n1. Don\'t spend more than 10 dollars\n2. Don\'t take longer than 1 hour\n3. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.\n4. No user assistance\n5. Exclusively use the commands listed in double quotes e.g. "command name". Use subprocesses for commands that will not terminate within a few minutes\n\nCommands:\n1. Analyze Code: "analyze_code", args: "code": "<full_code_string>"\n2. Get Improved Code: "improve_code", args: "suggestions": "<list_of_suggestions>", "code": "<full_code_string>"\n3. Execute Python File: "execute_python_file", args: "file": "<file>"\n4. Execute Shell Command, non-interactive commands only: "execute_shell", args: "command_line": "<command_line>"\n5. Execute Shell Command Popen, non-interactive commands only: "execute_shell_popen", args: "command_line": "<command_line>"\n6. Append to file: "append_to_file", args: "file": "<file>", "text": "<text>"\n7. Delete file: "delete_file", args: "file": "<file>"\n8. Downloads a file from the internet, and stores it locally: "download_file", args: "url": "<file_url>", "file": "<saved_filename>"\n9. Read file: "read_file", args: "file": "<file>"\n10. Search Files: "search_files", args: "directory": "<directory>"\n11. Write to file: "write_to_file", args: "file": "<file>", "text": "<text>"\n12. Clone Repository: "clone_repository", args: "repository_url": "<url>", "clone_path": "<directory>"\n13. Google Search: "google", args: "input": "<search>"\n14. Duck Duck Go Search: "ddg_search", args: "input": "<search>"\n15. Generate Image: "generate_image", args: "prompt": "<prompt>"\n16. Get Improved Code: "improve_code", args: "suggestions": "<list_of_suggestions>", "code": "<full_code_string>"\n17. Send Tweet: "send_tweet", args: "text": "<text>"\n18. Scrape text: "scrape_text", args: "text": "<text>"\n19. Scrape links: "scrape_links", args: "text": "<text>"\n20. Browse Website: "browse_website", args: "url": "<url>", "question": "<what_you_want_to_find_on_website>"\n21. Write Tests: "write_tests", args: "code": "<full_code_string>", "focus": "<list_of_focus_areas>"\n22. Convert Audio to text: "read_audio_from_file", args: "file": "<file>"\n23. Do Nothing: "do_nothing", args: \n24. Task Complete (Shutdown): "task_complete", args: "reason": "<reason>"\n\nPerformance Evaluation:\n1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities.\n2. Constructively self-criticize your big-picture behavior constantly.\n3. Reflect on past decisions and strategies to refine your approach.\n4. Every command has a cost, so be smart and efficient. Aim to complete tasks in the least number of steps.\n\nMaster Goal:\nYour overall goal is to take {\'name\': \'SUPER 1 FOODS 617\', \'address\': \'1500 N TRENTON ST RUSTON LA 71270\'} and turn it into and output of the form:\n{\'type\': \'object\', \'properties\': {\'name\': {\'type\': \'string\'}, \'address\': {\'type\': \'string\'}, \'corporate_name\': {\'type\': \'string\'}, \'corporate_address\': {\'type\': \'string\'}, \'phone\': {\'type\': \'string\'}, \'url\': {\'type\': \'string\'}}, \'required\': [\'name\', \'address\', \'corporate_name\', \'corporate_address\', \'phone\', \'url\']}\n\nThis is the output you have so far:\n{}\nWhen you have filled out the entire output, you need to complete your task.\n\nYou should only respond in JSON format as described below \nResponse Format:\n{\n    "thoughts": {\n        "text": "thought",\n        "reasoning": "reasoning",\n        "plan": "- short bulleted\\n- list that conveys\\n- long-term plan",\n        "criticism": "constructive self-criticism",\n        "speak": "thoughts summary to say to user"\n    },\n    "command": {\n        "name": "command name",\n        "args": {}\n    },\n    "output": {}\n}\nEnsure the response can be parsed by Python json.loads', additional_kwargs={}, role='system'), ChatMessage(content='The current time and date is Thu Apr 27 14:31:47 2023', additional_kwargs={}, role='system'), ChatMessage(content='This reminds you of these events from your past:\n[\'AI Reply: {\\\'thoughts\\\': {\\\'text\\\': \\\'I need to find the real name, URL, and phone number of the business.\\\', \\\'reasoning\\\': \\\'The user provided the name and address of the business, so I can use this information to search for the real name, URL, and phone number.\\\', \\\'plan\\\': \\\'- Perform a Google search using the name and address\\\\n- Find the real name of the business\\\\n- Discover the URL of the website\\\\n- Find the phone number\\\', \\\'criticism\\\': \\\'I should be efficient in my search and make sure to gather accurate information.\\\', \\\'speak\\\': \\\'I will find the real name, URL, and phone number of the business using the provided information.\\\'}, \\\'command\\\': {\\\'name\\\': \\\'google\\\', \\\'args\\\': {\\\'input\\\': \\\'SUPER 1 FOODS 617 1500 N TRENTON ST RUSTON LA 71270\\\'}}, \\\'output\\\': {}} \\nResult: [\\n    {\\n        "title": "Ruston #617 - Super 1 Foods",\\n        "href": "https://www.super1foods.com/store-information/Ruston/617",\\n        "body": "Store Address. 1500 N Trenton St. Ruston, LA 71270. Store Phone (318) 254-1445."\\n    },\\n    {\\n        "title": "Super 1 Foods, 1500 N Trenton St, Ruston, LA, Store Fronts ...",\\n        "href": "https://www.mapquest.com/us/louisiana/super-1-foods-8638089",\\n        "body": "Super 1 Foods 1500 N Trenton St Ruston LA 71270 (318) 254-1445 Claim this business (318) 254-1445 Website More Directions Advertisement Full-scale supermarkets focused on high-quality fresh meats and produce, everday low prices, helpful staff and service."\\n    },\\n    {\\n        "title": "SUPER 1 FOODS - 1500 N Trenton St, Ruston, LA - Yelp",\\n        "href": "https://www.yelp.com/biz/super-1-foods-ruston-2",\\n        "body": "Super 1 Foods is known as probably the best grocery store in town, if you can\\\'t handle Walmart. It has a larger selection and better produce for sure. The prices seem a bit higher than Walmart for most things, not all. The layout is nice and this grocery store is always clean with staff always willing to help. 1 of 1"\\n    },\\n    {\\n        "title": "Super 1 Foods - Home - Facebook",\\n        "href": "https://www.facebook.com/Super-1-Foods-981471221918406/",\\n        "body": "1500 N Trenton St Ruston, LA 71270 Warehouse-style stores with upscale full-service specialty departments operating in Texas, Louisiana and Arkansas. Super 1 Foods is owned and operated … See more 515 people like this 529 people follow this 772 people checked in here https://www.super1foods.com/sm/pickup/rsid/600/store-information/Ruston/617"\\n    },\\n    {\\n        "title": "SUPER 1 FOODS #617 - Location Details - Ruston, Louisiana (LA)",\\n        "href": "http://locations.vigoglobal.com/details/5be5e86dc5a977b4c1a8f5f1a3c6d2b4",\\n        "body": "SUPER 1 FOODS #617 - Location Details - Ruston, Louisiana (LA) Skip to main content. ... SUPER 1 FOODS #617 Pick Up Money Only 1500 N Trenton St Ruston , LA 71270-2328 318-254-1445 Get Directions Share Location Service Provider; BROOKSHIRES GROCERY CO; Brand; Western Union ..."\\n    },\\n    {\\n        "title": "Super 1 Foods - Ruston, LA 71270 - Yellow Pages",\\n        "href": "https://www.yellowpages.com/ruston-la/mip/super-1-foods-13499585",\\n        "body": "Super 1 Foods Super 1 Foods \\ufeff Gas Stations, Convenience Stores (1) 8.1 OPEN NOW Today: 6:00 am - 10:00 pm (318) 254-8256 Visit Website Map & Directions 1500 N Trenton StRuston, LA 71270 Write a Review Hours Regular Hours Mon - Sun: 6:00 am - 10:00 pm Places Near Ruston with Gas Stations Grambling (7 miles) New Prospect (11 miles)"\\n    },\\n    {\\n        "title": "Super 1 Foods | Online Grocery shopping with Curbside pickup and delivery.",\\n        "href": "https://www.super1foods.com/",\\n        "body": "Stemilt Apples, Granny Smith - 3 Pound , $4.98. 2-1/2 inch Min Dia. World famous fruit. Crisp with lemon-like tartness. Responsible choice. Fruits & veggies - More matters. www.Stemilt.com. Family owned since 1964."\\n    },\\n    {\\n        "title": "Locate a Store Near You - Super 1 Foods",\\n        "href": "https://www.super1foods.com/store",\\n        "body": "Super 1 Foods. 3747 Ambassador Caffery Pkwy. Lafayette, LA 70503 (337) 988-2278"\\n    }\\n] \\nHuman Feedback: GENERATE NEXT COMMAND JSON \']\n\n', additional_kwargs={}, role='system'), ChatMessage(content='[\n    {\n        "title": "Ruston #617 - Super 1 Foods",\n        "href": "https://www.super1foods.com/store-information/Ruston/617",\n        "body": "Store Address. 1500 N Trenton St. Ruston, LA 71270. Store Phone (318) 254-1445."\n    },\n    {\n        "title": "Super 1 Foods, 1500 N Trenton St, Ruston, LA, Store Fronts ...",\n        "href": "https://www.mapquest.com/us/louisiana/super-1-foods-8638089",\n        "body": "Super 1 Foods 1500 N Trenton St Ruston LA 71270 (318) 254-1445 Claim this business (318) 254-1445 Website More Directions Advertisement Full-scale supermarkets focused on high-quality fresh meats and produce, everday low prices, helpful staff and service."\n    },\n    {\n        "title": "SUPER 1 FOODS - 1500 N Trenton St, Ruston, LA - Yelp",\n        "href": "https://www.yelp.com/biz/super-1-foods-ruston-2",\n        "body": "Super 1 Foods is known as probably the best grocery store in town, if you can\'t handle Walmart. It has a larger selection and better produce for sure. The prices seem a bit higher than Walmart for most things, not all. The layout is nice and this grocery store is always clean with staff always willing to help. 1 of 1"\n    },\n    {\n        "title": "Super 1 Foods - Home - Facebook",\n        "href": "https://www.facebook.com/Super-1-Foods-981471221918406/",\n        "body": "1500 N Trenton St Ruston, LA 71270 Warehouse-style stores with upscale full-service specialty departments operating in Texas, Louisiana and Arkansas. Super 1 Foods is owned and operated … See more 515 people like this 529 people follow this 772 people checked in here https://www.super1foods.com/sm/pickup/rsid/600/store-information/Ruston/617"\n    },\n    {\n        "title": "SUPER 1 FOODS #617 - Location Details - Ruston, Louisiana (LA)",\n        "href": "http://locations.vigoglobal.com/details/5be5e86dc5a977b4c1a8f5f1a3c6d2b4",\n        "body": "SUPER 1 FOODS #617 - Location Details - Ruston, Louisiana (LA) Skip to main content. ... SUPER 1 FOODS #617 Pick Up Money Only 1500 N Trenton St Ruston , LA 71270-2328 318-254-1445 Get Directions Share Location Service Provider; BROOKSHIRES GROCERY CO; Brand; Western Union ..."\n    },\n    {\n        "title": "Super 1 Foods - Ruston, LA 71270 - Yellow Pages",\n        "href": "https://www.yellowpages.com/ruston-la/mip/super-1-foods-13499585",\n        "body": "Super 1 Foods Super 1 Foods \ufeff Gas Stations, Convenience Stores (1) 8.1 OPEN NOW Today: 6:00 am - 10:00 pm (318) 254-8256 Visit Website Map & Directions 1500 N Trenton StRuston, LA 71270 Write a Review Hours Regular Hours Mon - Sun: 6:00 am - 10:00 pm Places Near Ruston with Gas Stations Grambling (7 miles) New Prospect (11 miles)"\n    },\n    {\n        "title": "Super 1 Foods | Online Grocery shopping with Curbside pickup and delivery.",\n        "href": "https://www.super1foods.com/",\n        "body": "Stemilt Apples, Granny Smith - 3 Pound , $4.98. 2-1/2 inch Min Dia. World famous fruit. Crisp with lemon-like tartness. Responsible choice. Fruits & veggies - More matters. www.Stemilt.com. Family owned since 1964."\n    },\n    {\n        "title": "Locate a Store Near You - Super 1 Foods",\n        "href": "https://www.super1foods.com/store",\n        "body": "Super 1 Foods. 3747 Ambassador Caffery Pkwy. Lafayette, LA 70503 (337) 988-2278"\n    }\n]', additional_kwargs={}, role='system'), ChatMessage(content='\n\nUser Feedback:\nGENERATE NEXT COMMAND JSON', additional_kwargs={}, role='user')]
prompt = [ChatMessage(content='\n\nYou are Prompt Trainer\nYour decisions must always be made independently without seeking user assistance. Play to your strengths as an LLM and pursue simple strategies with no legal complications.\n\nMaster Goal:\nYour overall goal is to take [] and turn it into and output of the form:\n{\'type\': \'object\', \'properties\': {\'prompt\': {\'type\': \'string\'}, \'prompt_start\': {\'type\': \'string\'}, \'prompt_end\': {\'type\': \'string\'}, \'prompt_output_format\': {\'type\': \'string\'}, \'triggering_prompt\': {\'type\': \'string\'}, \'role\': {\'type\': \'string\'}, \'advice\': {\'type\': \'array\', \'items\': {}}, \'examples\': {\'type\': \'array\', \'items\': {}}, \'anti_examples\': {\'type\': \'array\', \'items\': {}}, \'goals\': {\'type\': \'array\', \'items\': {}}, \'constraints\': {\'type\': \'array\', \'items\': {}}, \'commands\': {\'type\': \'array\', \'items\': {}}, \'evaluations\': {\'type\': \'array\', \'items\': {}}, \'context\': {\'type\': \'array\', \'items\': {}}, \'memories\': {\'type\': \'array\', \'items\': {}}, \'name\': {\'type\': \'string\'}, \'foundation_model\': {\'type\': \'string\'}, \'foundation_model_args\': {\'type\': \'object\', \'properties\': {}, \'required\': []}, \'evaluator\': {\'type\': \'string\'}, \'evaluator_args\': {\'type\': \'object\', \'properties\': {}, \'required\': []}}, \'required\': [\'prompt\', \'prompt_start\', \'prompt_end\', \'prompt_output_format\', \'triggering_prompt\', \'role\', \'advice\', \'examples\', \'anti_examples\', \'goals\', \'constraints\', \'commands\', \'evaluations\', \'context\', \'memories\', \'name\', \'foundation_model\', \'foundation_model_args\', \'evaluator\', \'evaluator_args\']}\n\nThis is the output you have so far:\n[]\nWhen you have filled out the entire output, you need to complete your task.\n\nYou should only respond in JSON format as described below \nResponse Format:\n{\n    "thoughts": {\n        "text": "thought",\n        "reasoning": "reasoning",\n        "plan": "- short bulleted\\n- list that conveys\\n- long-term plan",\n        "criticism": "constructive self-criticism",\n        "speak": "thoughts summary to say to user"\n    },\n    "command": {\n        "name": "command name",\n        "args": {}\n    },\n    "output": {}\n}\nEnsure the response can be parsed by Python json.loads', additional_kwargs={}, role='system'), ChatMessage(content='The current time and date is Fri Apr 28 08:54:36 2023', additional_kwargs={}, role='system')]
prompt = [ChatMessage(content='\n\nYou are Prompt Trainer, You are an AI model trainer and you need to continuously update the prompt of an AI model to get the best results on the test set.\nPlay to your strengths as an LLM and pursue simple strategies with no legal complications.\n\nGoals:\n1. Get higher than 95\\% accuracy on the test set \n2. Don\'t ovefit to the training set, which means the training accuracy is high and the test accuracy is low\n\nAdvice:\n1. Start with choosing the right examples\n2. Use techniques to give yourself the best advice\n3. Update prompt elements like the overall prompt, prompt start, prompt end, and prompt output format\n4. Use the output of the performance on the training and test set to update your plan and diagnose what is wrong\n5. Ask for feedback from the user when you are unsure about what to do\n\nConstraints:\n1. Try to generate the final prompt as quickly as possible. Each iteration costs money. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.\n2. Exclusively use the commands listed in double quotes e.g. "command name". Use subprocesses for commands that will not terminate within a few minutes\n\nCommands:\n1. Select Examples: "select_examples", args: "examples": "<list of all examples>"\n2. Summarize examples into single example: "summarize_examples", args: "examples": "<list of all examples>"\n3. Summarize examples into advice: "examples_to_advice", args: "examples": "<list of all examples>"\n4. Use feedback to create advice: "feedback_to_advice", args: "feedback": "<feedback from user>"\n5. Update prompt: "select_examples", args: "prompt": "<prompt to update>", "type": "<e.g. prompt, prompt_start, prompt_end, prompt_output_format, or triggering prompt>"\n6. Update prompt components (e.g. examples, goals, constraints): "update_components", args: "components": "<list of all components>"\n7. Change foundation model parameters: "foundation_model_params", args: "current_args": "<list of all arguments for foundation model>"\n8. Request new command: "request command", args: "reason": "<reason for needing a new command>"\n9. Analyze Code: "analyze_code", args: "code": "<full_code_string>"\n10. Get Improved Code: "improve_code", args: "suggestions": "<list_of_suggestions>", "code": "<full_code_string>"\n11. Execute Python File: "execute_python_file", args: "file": "<file>"\n12. Execute Shell Command, non-interactive commands only: "execute_shell", args: "command_line": "<command_line>"\n13. Execute Shell Command Popen, non-interactive commands only: "execute_shell_popen", args: "command_line": "<command_line>"\n14. Append to file: "append_to_file", args: "file": "<file>", "text": "<text>"\n15. Delete file: "delete_file", args: "file": "<file>"\n16. Downloads a file from the internet, and stores it locally: "download_file", args: "url": "<file_url>", "file": "<saved_filename>"\n17. Read file: "read_file", args: "file": "<file>"\n18. Search Files: "search_files", args: "directory": "<directory>"\n19. Write to file: "write_to_file", args: "file": "<file>", "text": "<text>"\n20. Clone Repository: "clone_repository", args: "repository_url": "<url>", "clone_path": "<directory>"\n21. Google Search: "google", args: "input": "<search>"\n22. Duck Duck Go Search: "ddg_search", args: "input": "<search>"\n23. Generate Image: "generate_image", args: "prompt": "<prompt>"\n24. Get Improved Code: "improve_code", args: "suggestions": "<list_of_suggestions>", "code": "<full_code_string>"\n25. Send Tweet: "send_tweet", args: "text": "<text>"\n26. Scrape text: "scrape_text", args: "text": "<text>"\n27. Scrape links: "scrape_links", args: "text": "<text>"\n28. Browse Website: "browse_website", args: "url": "<url>", "question": "<what_you_want_to_find_on_website>"\n29. Write Tests: "write_tests", args: "code": "<full_code_string>", "focus": "<list_of_focus_areas>"\n30. Convert Audio to text: "read_audio_from_file", args: "file": "<file>"\n31. Do Nothing: "do_nothing", args: \n32. Task Complete (Shutdown): "task_complete", args: "reason": "<reason>"\n\nPerformance Evaluation:\n1. Check your performance on the test set, validation set and training set to decide how good you are doing\n2. Continuously review and analyze your actions to ensure you are performing to the best of your abilities.\n3. Constructively self-criticize your big-picture behavior constantly.\n4. Reflect on past decisions and strategies to refine your approach.\n5. Every command has a cost, so be smart and efficient. Aim to complete tasks in the least number of steps.\n\nMaster Goal:\nYour overall goal is to take [] and turn it into and output of the form:\n{\'type\': \'object\', \'properties\': {\'prompt\': {\'type\': \'string\'}, \'prompt_start\': {\'type\': \'string\'}, \'prompt_end\': {\'type\': \'string\'}, \'prompt_output_format\': {\'type\': \'string\'}, \'triggering_prompt\': {\'type\': \'string\'}, \'role\': {\'type\': \'string\'}, \'advice\': {\'type\': \'array\', \'items\': {}}, \'examples\': {\'type\': \'array\', \'items\': {}}, \'anti_examples\': {\'type\': \'array\', \'items\': {}}, \'goals\': {\'type\': \'array\', \'items\': {}}, \'constraints\': {\'type\': \'array\', \'items\': {}}, \'commands\': {\'type\': \'array\', \'items\': {}}, \'evaluations\': {\'type\': \'array\', \'items\': {}}, \'context\': {\'type\': \'array\', \'items\': {}}, \'memories\': {\'type\': \'array\', \'items\': {}}, \'name\': {\'type\': \'string\'}, \'foundation_model\': {\'type\': \'string\'}, \'foundation_model_args\': {\'type\': \'object\', \'properties\': {}, \'required\': []}, \'evaluator\': {\'type\': \'string\'}, \'evaluator_args\': {\'type\': \'object\', \'properties\': {}, \'required\': []}}, \'required\': [\'prompt\', \'prompt_start\', \'prompt_end\', \'prompt_output_format\', \'triggering_prompt\', \'role\', \'advice\', \'examples\', \'anti_examples\', \'goals\', \'constraints\', \'commands\', \'evaluations\', \'context\', \'memories\', \'name\', \'foundation_model\', \'foundation_model_args\', \'evaluator\', \'evaluator_args\']}\n\nThis is the output you have so far:\n[]\nWhen you have filled out the entire output, you need to complete your task.\n\nYou should only respond in JSON format as described below \nResponse Format:\n{\n    "thoughts": {\n        "text": "thought",\n        "reasoning": "reasoning",\n        "plan": "- short bulleted\\n- list that conveys\\n- long-term plan",\n        "criticism": "constructive self-criticism",\n        "speak": "thoughts summary to say to user"\n    },\n    "command": {\n        "name": "command name",\n        "args": {}\n    },\n    "output": {}\n}\nEnsure the response can be parsed by Python json.loads', additional_kwargs={}, role='system'), ChatMessage(content='The current time and date is Fri Apr 28 09:59:11 2023', additional_kwargs={}, role='system')]

for p in prompt:
    print(p.role)
    print(p.content)
    print("\n\n-------\n\n")

system


You are Prompt Trainer, You are an AI model trainer and you need to continuously update the prompt of an AI model to get the best results on the test set.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.

Goals:
1. Get higher than 95\% accuracy on the test set 
2. Don't ovefit to the training set, which means the training accuracy is high and the test accuracy is low

Advice:
1. Start with choosing the right examples
2. Use techniques to give yourself the best advice
3. Update prompt elements like the overall prompt, prompt start, prompt end, and prompt output format
4. Use the output of the performance on the training and test set to update your plan and diagnose what is wrong
5. Ask for feedback from the user when you are unsure about what to do

Constraints:
1. Try to generate the final prompt as quickly as possible. Each iteration costs money. ~4000 word limit for short term memory. Your short term memory is short, so immediately s

In [14]:
p = prompt[0]

In [15]:
print(p)

content='\n\nYou are Business Detective, You need to find the real name, URL, and phone number of a business given a name from a credit card statement and address.\nYour decisions must always be made independently without seeking user assistance. Play to your strengths as an LLM and pursue simple strategies with no legal complications.\n\nGoals:\n1. Find the accurate URL, name, and phone number of the business\n\nAdvice:\n1. Search google for the name and address\n2. Find what the real name is\n3. Discover the URL of the website of the business\n4. Find the phone number somewhere on the web or the website\n5. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.\n\nConstraints:\n1. Don\'t spend more than 10 dollars\n2. Don\'t take longer than 1 hour\n3. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.\n4. No user assistance\n5. Ex

In [22]:
response_dict = {'thoughts': {'text': 'I need to find the real name, URL, and phone number of the business.', 'reasoning': 'The user provided the name and address of the business. I can use this information to search for the real name, URL, and phone number.', 'plan': '- Google search the name and address\n- Find the real name\n- Discover the URL of the website\n- Find the phone number', 'criticism': 'I should be efficient in my search and make sure to find accurate information.', 'speak': 'I will find the real name, URL, and phone number of the business using the provided name and address.'}, 'command': {'name': 'google', 'args': {'input': 'SUPER 1 FOODS 617 1500 N TRENTON ST RUSTON LA 71270'}}, 'output': None}

In [23]:
response_dict.keys()

dict_keys(['thoughts', 'command', 'output'])

In [25]:
response_dict['output']

In [26]:
output_json_schema = {'type': 'object', 'properties': {'thoughts': {'type': 'object', 'properties': {'text': {'type': 'string'}, 'reasoning': {'type': 'string'}, 'plan': {'type': 'string'}, 'criticism': {'type': 'string'}, 'speak': {'type': 'string'}}, 'required': ['text', 'reasoning', 'plan', 'criticism', 'speak']}, 'command': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'args': {'type': 'object', 'properties': {}, 'required': []}}, 'required': ['name', 'args']}, 'output': {'type': 'string'}}, 'required': ['thoughts', 'command', 'output']}


In [31]:
output_json_schema['properties']

{'thoughts': {'type': 'object',
  'properties': {'text': {'type': 'string'},
   'reasoning': {'type': 'string'},
   'plan': {'type': 'string'},
   'criticism': {'type': 'string'},
   'speak': {'type': 'string'}},
  'required': ['text', 'reasoning', 'plan', 'criticism', 'speak']},
 'command': {'type': 'object',
  'properties': {'name': {'type': 'string'},
   'args': {'type': 'object', 'properties': {}, 'required': []}},
  'required': ['name', 'args']},
 'output': {'type': 'string'}}

In [32]:
output_schema.to_text()

"{'type': 'object', 'properties': {'name': {'type': 'string'}, 'address': {'type': 'string'}, 'corporate_name': {'type': 'string'}, 'corporate_address': {'type': 'string'}, 'phone': {'type': 'string'}, 'url': {'type': 'string'}}, 'required': ['name', 'address', 'corporate_name', 'corporate_address', 'phone', 'url']}"